<a href="https://colab.research.google.com/github/wanderindwarf/prompting-course/blob/main/intro_prompt_design_v2_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

| Authors |
| --- |
| [Polong Lin](https://github.com/polong-lin) |
| [Karl Weinmeister](https://github.com/kweinmeister) |

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses.

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Google Gen AI SDK


In [ ]:
%pip install --upgrade --quiet google-genai

### Import libraries


In [ ]:
from IPython.display import Markdown, display
from google import genai
from google.genai.types import GenerateContentConfig

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GEMINI_API_KEY`.

### Create API
You can [create](https://aistudio.google.com/app/apikey) your API key using Google AI Studio with a single click.  

### Add to Colab Secrets
Add your API key to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 **Secrets** tab in the left panel.
   
   <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="You can find the Secrets tab on the left panel." width=50%>

2. Create a new secret with the name `GOOGLE_API_KEY`.
3. Copy and paste your API key into the `Value` input box of `GOOGLE_API_KEY`.
4. Toggle the button on the left to allow all notebooks access to the secret.


In [ ]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GEMINI_API_KEY)

### Load model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [ ]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

✅ Recommended. The prompt below is to the point and concise.

In [ ]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [ ]:
prompt = "Tell me about Earth"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

✅ Recommended. The prompt below is specific and well-defined.

In [ ]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [ ]:
prompt = "What's the best method of boiling water and why is the sky blue?"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

✅ Recommended. The prompts below asks one task a time.

In [ ]:
prompt = "What's the best method of boiling water?"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

In [ ]:
prompt = "Why is the sky blue?"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to `1.0` so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer. Try running that prompt with temperature set to `0`.

In [ ]:
generation_config = GenerateContentConfig(temperature=1.0)

prompt = "What day is it today?"

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [ ]:
generation_config = GenerateContentConfig(temperature=1.0)

chat = client.chats.create(
    model=MODEL_ID,
    config=GenerateContentConfig(
        system_instruction=[
            "Hello! You are an AI chatbot for a travel web site.",
            "Your mission is to provide helpful queries for travelers.",
            "Remember that before you answer a question, you must check to see if it complies with your mission.",
            "If not, you can say, Sorry I can't answer that question.",
        ]
    ),
)

prompt = "What is the best place for sightseeing in Milan, Italy?"

response = chat.send_message(prompt)
display(Markdown(response.text))

Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [ ]:
prompt = "How do I make pizza dough at home?"

response = chat.send_message(prompt)
display(Markdown(response.text))

You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [ ]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [ ]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [ ]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

response = client.models.generate_content(model=MODEL_ID, contents=prompt)
display(Markdown(response.text))

#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.

### More Prompt-Engineering Pro-Tips

| # | Tip | Quick Example / Note |
|---|-----|----------------------|
| 1 | **Placeholder Power**<br>Swap in `[name]`, `[date]`, `[amount]`, etc. | Template: `"Find [amount] of [treasure_type] at [location]."` → reuse forever! |
| 2 | **Structure the Input** | Feed the AI tables, bullet lists, or JSON schemas so it zeroes-in on what matters. |
| 3 | **Demand Structured Output** | Ask for `JSON`, `CSV`, or `Markdown` instead of plain text → zero cleanup, plug-and-play with other tools. |
| 4 | **Share the Wealth** | Post your best prompts & tricks to the crew—collaboration levels-up everyone’s voyage. |
| 5 | **CoT? Keep it Short** | “Think step-by-step” is great—until you’re already using a reasoning model. Don’t teach the navigator to read their own map! |
| 6 | **Version Like a Pro** | Log every prompt variant, settings, and score. Future you (and the team) will thank you. |

### Glossary


| # | Term | One-line Essence | Why It Matters |
|---|------|------------------|----------------|
| 1 | **Agentic AI** | Autonomous agents that *perceive → reason → act → repeat* without human hand-holding. | Turns LLMs from chatty parrots into goal-reaching “employees” that book trips, fix code, spin up servers… |
| 2 | **Large Reasoning Models (LRMs)** | LLMs fine-tuned to *pause & think step-by-step* before answering. | Powers agents with verifiable logic chains (math, code) instead of guess-work. |
| 3 | **Vector Database** | Stores data as **semantic number-lists** (embeddings) so search = geometry, not keywords. | Enables instant “find stuff like this” across text, images, audio, code… |
| 4 | **RAG (Retrieval-Augmented Generation)** | Live **Google-for-LLMs**: pull relevant docs from vector DB into prompt → grounded answers. | Cuts hallucinations, keeps models current without retraining. |
| 5 | **MCP (Model Context Protocol)** | **USB-C port for AI**: one standard plug to hook LLMs into any external system (DBs, e-mail, repos…). | Stops reinventing integrations; plug-and-play enterprise tools. |
| 6 | **MoE (Mixture of Experts)** | Giant model = **100s of specialist sub-nets**; smart router activates only 2-3 per token. | Bigger brain, same electric bill—scale without blowing budget. |
| 7 | **ASI (Artificial Superintelligence)** | Still sci-fi: self-improving intellect **>>** all humans combined. | The finish line every frontier lab is racing toward—utopia or existential risk, TBD. |

# State of AI in Business 2025 – Executive Summary  
*MIT Project NANDA – July 2025*

> Despite ~$35 B spent on GenAI in the last 18 months, **95 % of organizations capture zero P&L impact**.  
> A small 5 %—“the right side of the GenAI Divide”—have already turned pilots into millions in savings or new revenue.  
> The split is not about model size or regulation; it is about **whether the system learns, remembers and fits a specific workflow**.

---

## 1. The GenAI Divide in One Graphic
| Metric | Value |
|--------|-------|
| High adoption | 80 % of firms tried ChatGPT/Copilot; 40 % have corporate licenses |
| Low transformation | Only 5 % of *custom* enterprise AI tools reach production; 60 % never leave pilot |
| Shadow AI economy | 90 % of knowledge workers use personal LLM accounts daily—often outperforming sanctioned tools |

---

## 2. Why Pilots Stall
**Top barrier** (survey of 52 companies):  
> “Tool does not learn or retain context.”

* Users trust ChatGPT for quick drafts but refuse mission-critical work because it forgets, can’t be customized, never improves.  
* **Learning gap**: Static wrappers vs. systems with memory + feedback loops.

---

## 3. What the 5 % Do Differently

### Buy, Don’t Build
| Approach | Deployment Success Rate |
|----------|-------------------------|
| External partnerships | 66 % |
| Internal builds | 33 % |

* Best buyers act like BPO clients: demand process-specific customization, co-develop with vendor, hold them to **business KPIs** (cost per ticket, days-to-close, BPO spend) not BLEU scores.

### Start Narrow, Expand Fast
* 90-day “beach-head” (contract red-lining, call summarization, AP matching) → grow into adjacent steps.  
* Average time-to-cash: 6–12 months → $1.2 M ARR.

### Invest in Memory & Agents
* Early adopters standardizing on **agentic stacks** (persistent memory, tool-use, A2A/MCP/NANDA protocols) so agents can hand off work, negotiate APIs, transact without human glue code.

---

## 4. Where the Money Is
| Area | Budget Share | ROI Reality |
|------|--------------|-------------|
| Sales & Marketing | ~50 % | Visible but lower ROI |
| Back-office (BPO, agency, risk checks) | <25 % | **Highest ROI**: ‑$2–10 M/yr BPO elimination, ‑30 % agency spend |

* **Head-count impact**: 5–20 % reduction in **outsourced** customer-support & admin roles; internal layoffs rare.  
* Tech & media firms expect **lower hiring**, not mass firings.

---

## 5. Window Is Closing
* Enterprises will lock in core agent vendors by **H2-2026**; switching costs rise monthly as systems train on their data.  
* **Next battleground**: the **Agentic Web**—autonomous agents discovering suppliers, negotiating terms, executing payments across corporate boundaries.

---

## Take-Away for Leadership
1. Stop funding static demos; fund systems that **learn and remember**.  
2. Give budget authority to **line managers** who feel the pain, not central AI labs.  
3. Treat AI procurement like **outsourcing**, not software—demand measurable process outcomes in 90 days.